<p align="center">
  <img src="https://github.com/vpj/lab/raw/832758308905ee20ba9841fa80c47c77d7e58fda/images/logo.png?raw=true" width="100" title="Logo">
</p>

# [Lab 3.0](https://github.com/vpj/lab)

This library helps you organize machine learning experiments.
It is a quite small library,
 and most of the modules can be used independently of each other.
This doesn't have any user interface.
Experiment results are maintained in a folder structure,
and there is a Python API to access them.

## Features

### Organize Experiments

Maintains logs, summaries and checkpoints of all the experiment runs in a folder structure.

```
logs
├── experiment1
│   ├── run1
│   │   ├── run.yaml
│   │   ├── configs.yaml
│   │   ├── indicators.yaml
│   │   ├── source.diff
│   │   ├── checkpoints
│   │   │   └── 📄 Saved checkpoints
│   │   └── tensorboard
│   │       └── 📄 TensorBoard summaries
│   ├── run1
│   ...
└── experiment2...
    ├──
    ...
```

### [🎛 Dashboard](https://github.com/vpj/lab_dashboard) to browse experiments
<p align="center">
  <img style="max-width:100%;" src="https://raw.githubusercontent.com/vpj/lab/master/images/dashboard.png" width="320" title="Logo">
</p>

The web dashboard helps navigate experiments and multiple runs.
You can checkout the configs and a summary of performance.
You can launch TensorBoard directly from there.

Eventually, we want to let you edit configs and run new experiments and analyse
outputs on the dashboard.

### Logger

Logger has a simple API to produce pretty console outputs.

<p align="center"><img style="max-width:100%" src="https://github.com/vpj/lab/raw/832758308905ee20ba9841fa80c47c77d7e58fda/images/loop.gif" /></p>

### Manage configurations and hyper-parameters

You can setup configs/hyper-parameters with functions.
[🧪lab](https://github.com/vpj/lab) would identify the dependencies and run 
them in topological order.

```python
@Configs.calc()
def model(c: Configs):
    return Net().to(c.device)
```

You can setup multiple options for configuration functions. 
So you don't have to write a bunch if statements to handle configs.

```python
@Configs.calc('optimizer')
def sgd(c: Configs):
    return optim.SGD(c.model.parameters(), lr=c.learning_rate, momentum=c.momentum)

@Configs.calc('optimizer')
def adam(c: Configs):
    return optim.Adam(c.model.parameters())
```

## Getting Started

### Clone and install

```bash
git clone git@github.com:vpj/lab.git
cd lab
pip install -e .
```

To update run a git update

```bash
cd lab
git pull
```

<!-- ### Install it via `pip` directly from github.

```bash
pip install -e git+git@github.com:vpj/lab.git#egg=lab
``` -->

### Create a `.lab.yaml` file.
An empty file at the root of the project should
be enough. You can set project level configs for
 'check_repo_dirty' and 'path'
in the config file.

Lab will store all experiment data in folder `logs/` 
relative to `.lab.yaml` file.
If `path` is set in `.lab.yaml` then it will be stored in `[path]logs/`
 relative to `.lab.yaml` file.

You don't need the `.lab.yaml` file if you only plan on using the logger.

### [Samples](https://github.com/vpj/lab/tree/master/samples)

[Samples folder](https://github.com/vpj/lab/tree/master/samples) contains a
 bunch of examples of using 🧪 lab.

## Tutorial

*The outputs lose color when viewing on github. Run [readme.ipynb](https://github.com/vpj/lab/blob/master/readme.ipynb) locally to try it out.*

This short tutorial covers most of the usage patterns. We still don't have a proper documentation, but the source code of the project is quite clean and I assume you can dive into it if you need more details.

In [ ]:
# Some imports
import numpy as np
import time

### Logger

In [ ]:
from lab import logger
from lab.logger.colors import Text, Color

#### Logging with colors

In [ ]:
logger.log("Colors are missing when views on github", Text.highlight)

logger.log([
    ('Styles\n', Text.heading),
    ('Danger\n', Text.danger),
    ('Warning\n', Text.warning),
    ('Meta\n', Text.meta),
    ('Key\n', Text.key),
    ('Meta2\n', Text.meta2),
    ('Title\n', Text.title),
    ('Heading\n', Text.heading),
    ('Value\n', Text.value),
    ('Highlight\n', Text.highlight),
    ('Subtle\n', Text.subtle)
])

logger.log([
    ('Colors\n', Text.heading),
    ('Red\n', Color.red),
    ('Black\n', Color.black),
    ('Blue\n', Color.blue),
    ('Cyan\n', Color.cyan),
    ('Green\n', Color.green),
    ('Orange\n', Color.orange),
    ('Purple Heading\n', [Color.purple, Text.heading]),
    ('White\n', Color.white),
])

##### Logging debug info

In [ ]:
logger.info(a=2, b=1)
logger.info(dict(name='Name', price=22))

### Sections

Sections let you monitor time taken for
different tasks and also helps *keep the code clean*
by separating different blocks of code.

In [ ]:
with logger.section("Load data"):
    # code to load data
    time.sleep(2)

with logger.section("Load saved model"):
    time.sleep(1)
    logger.set_successful(False)
    # code to create model

#### Progress

This shows the progress for code within the section.

In [ ]:
with logger.section("Train", total_steps=100):
    for i in range(100):
        time.sleep(0.1)
        # Multiple training steps in the inner loop
        logger.progress(i)

### Iterator and Enumerator

This combines `section` and `progress`. In this example we use a PyTorch `DataLoader`. You can use `logger.iterate` and `logger.enumerate` with any iterable object.

In [ ]:
# Create a data loader for illustration
import torch
from torchvision import datasets, transforms

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data',
                       train=False,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=32, shuffle=True)

In [ ]:
for data, target in logger.iterate("Test", test_loader):
    time.sleep(0.01)

In [ ]:
for i, (data, target) in logger.enum("Test", test_loader):
    time.sleep(0.01)

### Loop

The `loop` keeps track of the time taken and time remaining for the loop.
You can use *sections*, *iterators* and *enumerators* within loop.

`logger.write` outputs the current status along with global step.

In [ ]:
for step in logger.loop(range(0, 400)):
	logger.write()

#### Global step

The global step is used for logging to the screen, TensorBoard and when logging analytics to SQLite. You can manually set the global step. Here we will reset it.

In [ ]:
logger.set_global_step(0)

You can manually increment global step too.

In [ ]:
for step in logger.loop(range(0, 400)):
    logger.add_global_step(5)
    logger.write()

### Log indicators

Here you specify indicators and the logger stores them temporarily and write in batches.
It can aggregate and write them as means or histograms.

In [ ]:
# dummy train function
def train():
    return np.random.randint(100)

# Reset global step because we incremented in previous loop
logger.set_global_step(0)

This stores all the loss values and writes the logs the mean on every tenth iteration.
Console output line is replaced until `new_line` is called.

In [ ]:
for i in range(1, 401):
    logger.add_global_step()
    loss = train()
    logger.store(loss=loss)
    if i % 10 == 0:
        logger.write()
    if i % 100 == 0:
        logger.new_line()
    time.sleep(0.02)

#### Indicator settings

In [ ]:
from lab.logger.indicators import Queue, Scalar, Histogram

In [ ]:
# dummy train function
def train2(idx):
    return idx, 10, np.random.randint(100)

# Reset global step because we incremented in previous loop
logger.set_global_step(0)

`histogram` indicators will log a histogram of data.
`queue` will store data in a `deque` of size `queue_size`, and log histograms.
Both of these will log the means too. And if `is_print` is `True` it will print the mean.

In [ ]:
# queue_size = 10, 
logger.add_indicator(Queue('reward', 10, True))
# is_print default to False
logger.add_indicator(Scalar('policy'))
# is_print = True
logger.add_indicator(Histogram('value', True))

In [ ]:
for i in range(1, 400):
    logger.add_global_step()
    reward, policy, value = train2(i)
    logger.store(reward=reward, policy=policy, value=value, loss=1.)
    if i % 10 == 0:
        logger.write()
    if i % 100 == 0:
        logger.new_line()

### Experiment

Lab will keep track of experiments if you declare an Experiment. It will keep track of logs, code diffs, git commits, etc.

In [ ]:
from lab.experiment.pytorch import Experiment

In [ ]:
exp = Experiment(name="mnist_pytorch",
                 comment="Test")

The `name` of the defaults to the calling python filename. However when invoking from a Jupyter Notebook it must be provided because the library cannot find the calling file name.

#### Starting an expriemnt

This creates the folders, stores the experiment meta data, git commits, and source diffs.

In [ ]:
exp.start()

You can also start from a previously saved checkpoint. A `run_index` of `-1` means that it will load from the last run.

```python
exp.start(run_index=-1)
```

#### Save Checkpoint

In [ ]:
logger.save_checkpoint()

### Configs

In [ ]:
from lab import configs

The configs will be stored and in future be adjusted from  [🎛 Dashboard](https://github.com/vpj/lab_dashboard)

In [ ]:
class DeviceConfigs(configs.Configs):
    use_cuda: bool = True
    cuda_device: int = 0

    device: any

Some configs can be calculated

In [ ]:
import torch

In [ ]:
@DeviceConfigs.calc('device')
def cuda(c: DeviceConfigs):
    is_cuda = c.use_cuda and torch.cuda.is_available()
    if not is_cuda:
        return torch.device("cpu")
    else:
        if c.cuda_device < torch.cuda.device_count():
            return torch.device(f"cuda:{c.cuda_device}")
        else:
            logger.log(f"Cuda device index {c.cuda_device} higher than "
                       f"device count {torch.cuda.device_count()}", Text.warning)
            return torch.device(f"cuda:{torch.cuda.device_count() - 1}")

Configs classes can be inherited

In [ ]:
class Configs(DeviceConfigs):
    model_size: int = 10
        
    model: any = 'cnn_model'

You can specify multiple config calculator functions. The function given by the string for respective attribute will be picked.

In [ ]:
@Configs.calc('model')
def cnn_model(c: Configs):
    return c.model_size * 10

@Configs.calc('model')
def lstm_model(c: Configs):
    return c.model_size * 2

The experiment will calculate the configs.

In [ ]:
conf = Configs()
conf.model = 'lstm_model'
experiment = Experiment(name='test_configs')
experiment.calc_configs(conf)
logger.info(model=conf.model)

---

## Background
I was coding existing reinforcement learning algorithms
 to play Atari games for fun.
It was not easy to keep track of things when I started
 trying variations, fixing bugs etc.
Then I wrote some tools to organize my experiment runs.
I found it important to keep track of git commits
to make sure I can reproduce results.

I also wrote a logger to display pretty results on screen and
 to make it easy to write TensorBoard summaries.
It also keeps track of training times which makes it easy to spot
 what's taking up most resources.

This library is was made by combining these bunch of tools.

## Alternatives

### Managing Experiments

* [Comet](https://www.comet.ml/)
* [Beaker](https://beaker.org/)
* [Sacred](https://github.com/IDSIA/sacred)
* [Neptune](https://neptune.ml/)
* [Model Chimp](https://www.modelchimp.com/)

### Logging

* [TQDM](https://tqdm.github.io/)
* [Loguru](https://github.com/Delgan/loguru)